In [1]:
import requests
import json
import random
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

PATH_TO_IMG = "IMG_20201210_160628.jpg"
PATH_TO_MODEL = "../models/RV2-NASNetMobile-non-trainable"


In [2]:
# load model
model = load_model(PATH_TO_MODEL)


In [3]:
# load image and process
def get_proc(path):
    width = 75
    height = 150
    
    img = Image.open(path)
    img = img.resize((width, height), resample=1)
    img = img_to_array(img)
    img = img/255
    img = img.astype('float32')
    img = np.expand_dims(img, axis=0)
    
    return img
    
# prediction
def get_pred(img):
    return model.predict(img)[0,0]


# post_process
def post_process(pred):
    labels = ['good', 'kyphosis']
    threshold = 0.5
    
    if pred >= threshold:
        index = 1
        
    else:
        index = 0
        
    return labels[index]


def pipe(path):
    img = get_proc(path)
    pred = get_pred(img)
    label = post_process(pred)
    
    return label

In [78]:
APPARATUS = "Bodyweight"
PART_OF_BODY = "Shoulders"


def get_token(username,password):
    response = requests.post('http://204.235.60.194/consumer/login', data={'username':username, 'password':password})
    token =response.json()
    return token


def request(param, token, n_exercise):
    url = 'http://204.235.60.194/exrxapi/v1/allinclusive/exercises?'
    keys_to_extract = ['Apparatus_Name','Exercise_Name_Complete_Abbreviation','Small_Img_1','Small_Img_2','Larg_Img_1',\
                       'Instructions_Preparation','Instructions_Execution','video_src','GIF_Img']
    
    response = requests.get(url, params=param, headers={'Authorization': "Bearer "+token['token']})
    response = json.loads(response.text)['exercises']
    
    exercices = np.random.choice(response,n_exercise,replace=False)
    ex_list = []
    for exercice in exercices:
        ex_list.append(dict( (key, value) for (key, value) in exercice.items() if key in keys_to_extract))

    return ex_list
        
    
    
def main(apparatus, part_of_body):
    token = get_token('raneenaexrx','StKhg6Zu')
    
    stretch_params = {'stretch':'', 'bodypart':[]}
    strength_params = {'bodypart':[], 'apparatus':[]}
    
    stretch_params['bodypart'].append(part_of_body)
    strength_params['bodypart'].append(part_of_body)
    strength_params['apparatus'].append(apparatus)

    
    response_stretch = request(stretch_params, token,1)
    response_strength = request(strength_params, token,2)
    
    return response_stretch, response_strength
    
    
def activate():
    if pipe(PATH_TO_IMG) == 'kyphosis':
        return main(APPARATUS, PART_OF_BODY)

In [79]:
stretch, strength = activate()
